In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation, preprocessing, metrics

In [19]:
df_train = pd.read_csv('aggregated_train_with_nummosquitos.csv')# from make_aggregated_with_Nummosquitos
#df_test = pd.read_csv('../test_filled_new.csv')
#orig = pd.read_csv('../test_filled_new.csv')
df_test = pd.read_csv('../test_filled_new.csv')
train_support = pd.read_csv('../train.csv')
test_support = pd.read_csv('../test.csv')
#df_train['WnvPresent'] = train_support['WnvPresent']
#df_train.Date = train_support.Date
#df_test.Date = test_support.Date
df_trainSpecies = train_support.Species
df_test.Species = test_support.Species

In [20]:
df_train['counter'] = 1

In [21]:
X = df_train.copy()


In [22]:
string_features = ['Address','Street','AddressNumberAndStreet','CodeSum']
species_encoder = preprocessing.LabelEncoder()
#trap_encoder = preprocessing.LabelEncoder()
X_dump = X[string_features]
df_dump = df_test[string_features]
#X_dump = X[string_features]

df_test.Species[df_test.Species == 'UNSPECIFIED CULEX'] = 'CULEX PIPIENS/RESTUANS'
#X.Species = species_encoder.fit_transform(X.Species)
#y = df_train['NumMosquitos']
#df_train.Species = species_encoder.transform(df_train.Species)
X = X.drop(string_features,axis=1)

/home/thomas/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
groups = ['Date','Trap','Species']

In [24]:
tmp = X.groupby(groups)['NumMosquitos'].sum()


In [25]:
X_tmp = X.join(X.groupby(groups)['NumMosquitos'].sum(), on= groups,rsuffix='_counter')

In [26]:
X_tmp.NumMosquitos_counter.max()

2532

In [27]:
y = X_tmp.NumMosquitos_counter


In [28]:
#X_tmp.drop('Date',axis=1)
#X_tmp.drop('Trap',axis = 1)#del X.Trap
del X_tmp['Trap']
del X_tmp['Date']
del X_tmp['NumMosquitos']
del X_tmp['NumMosquitos_counter']
del X_tmp['WnvPresent']
del X_tmp['counter']
del X_tmp['Unnamed: 0']
#['Unnamed: 0', 'WnvPresent', 'counter']

X_tmp.Species = species_encoder.fit_transform(X.Species)
df_test.Species = species_encoder.transform(df_test.Species)
df_train.species = species_encoder.transform(df_train.Species)

In [29]:
X = X_tmp.copy()

In [30]:
from sklearn.cross_validation import train_test_split, ShuffleSplit,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, SGDRegressor
import numpy as np

In [31]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation, preprocessing, metrics
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [32]:

clf_list= [#RandomForestRegressor(n_estimators=100,n_jobs=-1),
           #RandomForestRegressor(n_estimators=400,n_jobs=-1),
           #RandomForestRegressor(n_estimators=1000,n_jobs=-1), 
           #Ridge(alpha = .01,normalize=True),
           #Ridge(alpha = .1,normalize=True),
           #Ridge(alpha = .1),
           #GradientBoostingRegressor(n_estimators = 100,max_depth=7),
          # GradientBoostingRegressor(n_estimators = 1000,max_depth=10),
          # GradientBoostingRegressor(n_estimators = 1000,max_depth=15,subsample = .7),#so far the best one, n_es = 400
           GradientBoostingRegressor(n_estimators = 40,max_depth=15,learning_rate = 0.01),
          # GradientBoostingRegressor(n_estimators = 1000,max_depth=20,subsample = .7)
           #KNeighborsRegressor(),
           #SGDRegressor()
           #GradientBoostingRegressor(n_estimators = 100,max_depth=4),
           #GradientBoostingRegressor(n_estimators = 100,max_depth=2)
      ]
random_state= 46


In [33]:
X.dtypes

Species                      int64
Block                        int64
Latitude                   float64
Longitude                  float64
AddressAccuracy              int64
Calendar_Week                int64
Station                      int64
Tmax                         int64
Tmin                         int64
Tavg                       float64
Depart                     float64
DewPoint                     int64
WetBulb                    float64
Heat                         int64
Cool                         int64
Sunrise                    float64
Sunset                     float64
Depth                      float64
SnowFall                   float64
PrecipTotal                float64
StnPressure                float64
SeaLevel                   float64
ResultSpeed                float64
ResultDir                    int64
AvgSpeed                   float64
Year                         int64
Month                        int64
Tavg_week                  float64
precip_week         

In [34]:
X = scale(X)


err = []
r2 = []
#get lengths of y_arr -.-

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=.33, random_state= random_state)

y_arr = pd.DataFrame(y_test)
i = 1
#clf = clf_list[1]
for clf in clf_list:
#for i in np.random.randint(1,size=4):
    i += 1
    X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=.33, random_state= random_state)
    clf.fit(X_train,y_train)
    yhat = clf.predict(X_test)
    err.append( mean_squared_error(yhat,y_test))
    r2.append(r2_score(yhat,y_test))
    y_arr[i] = yhat
    

In [35]:
y_arr_mean = y_arr.drop(0,axis=1)
y_arr_mean = y_arr_mean.mean(axis=1)
y_arr_mean
r2_score(y_arr_mean,y_arr[0])

-3.0912801856204553

In [36]:
r2

[-3.0912801856204553]

In [37]:
X_tmp

,Species,Block,Latitude,Longitude,AddressAccuracy,Calendar_Week,Station,Tmax,Tmin,Tavg,...,19_week_avrgPrecipTotal,19_week_avrgTavg,20_week_avrgPrecipTotal,20_week_avrgTavg,21_week_avrgPrecipTotal,21_week_avrgTavg,22_week_avrgPrecipTotal,22_week_avrgTavg,23_week_avrgPrecipTotal,23_week_avrgTavg
0,2,41,41.954690,-87.800991,9,22,1,88,60,74,...,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862
1,3,41,41.954690,-87.800991,9,22,1,88,60,74,...,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862
2,3,62,41.994991,-87.769279,9,22,1,88,60,74,...,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862
3,2,79,41.974089,-87.824812,8,22,1,88,60,74,...,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862
4,3,79,41.974089,-87.824812,8,22,1,88,60,74,...,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862
5,2,65,41.999129,-87.795585,8,22,1,88,60,74,...,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862
6,2,75,42.017430,-87.687769,8,22,1,88,60,74,...,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862,0.062586,63.275862
7,3,15,41.921600,-87.666455,8,22,2,88,65,77,...,0.043621,64.428571,0.043621,64.428571,0.043621,64.428571,0.043621,64.428571,0.043621,64.428571
8,3,25,41.891118,-87.654491,8,22,2,88,65,77,...,0.043621,64.428571,0.043621,64.428571,0.043621,64.428571,0.043621,64.428571,0.043621,64.428571
9,2,11,41.867108,-87.654224,8,22,2,88,65,77,...,0.043621,64.428571,0.043621,64.428571,0.043621,64.428571,0.043621,64.428571,0.043621,64.428571


In [38]:
df_train['count_mosquitos'] = y

In [39]:
clf.fit(X,y)

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.01, loss='ls',
             max_depth=15, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2, n_estimators=40,
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

In [40]:
df_test = df_test.drop(string_features,axis=1)


In [41]:
[var for var in df_test.columns if var not in X_tmp.columns]

['Id', 'Date', 'Trap']

In [42]:
[var for var in X_tmp.columns if var not in df_test.columns]

[]

In [43]:
df_test = df_test.drop(['Id', 'Date', 'Trap'],axis=1)

In [44]:
[var for var in df_test.columns if var not in X_tmp.columns]

[]

In [45]:
[var for var in X_tmp.columns if var not in df_test.columns]

[]

In [46]:
[var for var in X_tmp.columns if var not in df_train.columns]

[]

In [47]:
[var for var in df_train.columns if var not in X_tmp.columns]

['Unnamed: 0',
 'Date',
 'Address',
 'Street',
 'Trap',
 'AddressNumberAndStreet',
 'NumMosquitos',
 'WnvPresent',
 'CodeSum',
 'counter',
 'count_mosquitos']

In [48]:
df_test2 = scale(df_test)

In [49]:
yhat = clf.predict(df_test2)

In [50]:
yhat.mean(),yhat.std(),y.mean(),y.std()

(79.557504461165479,
 42.422355971664302,
 101.51913192461451,
 357.79678642553256)

In [51]:
yhat

array([ 69.90137192,  69.90137192,  69.47189092, ...,  72.37726765,
        72.41131826,  72.41131826])

In [52]:
df_test['count_mosquitos'] = yhat

In [53]:
[var for var in df_train if var not in df_test]

['Unnamed: 0',
 'Date',
 'Address',
 'Street',
 'Trap',
 'AddressNumberAndStreet',
 'NumMosquitos',
 'WnvPresent',
 'CodeSum',
 'counter']

In [54]:
strf = ['Unnamed: 0',
 'Date',
 'Address',
 'Street',
 'Trap',
 'AddressNumberAndStreet',
 'NumMosquitos',
 #'WnvPresent',
 'CodeSum',
 'counter']

In [55]:
df_train = df_train.drop(strf,axis=1)

In [56]:
[var for var in df_train if var not in df_test]

['WnvPresent']

In [57]:
[var for var in df_test if var not in df_train]

[]

In [58]:
train_support = pd.read_csv('../train.csv')
test_support = pd.read_csv('../test.csv')
df_train['WnvPresent'] = train_support['WnvPresent']
df_train.Date = train_support.Date
df_test.Date = test_support.Date
df_trainSpecies = train_support.Species
df_test.Species = test_support.Species

In [59]:
df_test.to_csv('new_test.csv',index_label='Id')
df_train.to_csv('new_train.csv',index_label='Id')

In [60]:
df_train.WnvPresent

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
10476    0
10477    0
10478    0
10479    0
10480    0
10481    0
10482    0
10483    0
10484    1
10485    0
10486    0
10487    0
10488    0
10489    0
10490    0
10491    0
10492    0
10493    0
10494    0
10495    0
10496    0
10497    0
10498    0
10499    0
10500    0
10501    1
10502    0
10503    0
10504    0
10505    0
Name: WnvPresent, dtype: int64